In [5]:
# import libraries
try:
  %tensorflow_version
except Exception:
  pass
!pip install tensorflow
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds (not used)
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 827.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


2.20.0


In [6]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2026-01-02 06:05:05--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2026-01-02 06:05:06 (22.7 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2026-01-02 06:05:06--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [7]:
# First i created training and testing dataframe
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=['label','message'])

# I mapped every ham as 0 and spam as 1 in both the dataframes
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

from tensorflow.keras.preprocessing.text import Tokenizer

VOCAB_SIZE = 10000

# Tokenized every word, and characterized unseen words ( if occurs during prediction) as <OOV> format token
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['message'])

train_sequences = tokenizer.texts_to_sequences(train_data['message'])
test_sequences = tokenizer.texts_to_sequences(test_data['message'])

train_labels = train_data['label'].values
test_labels = test_data['label'].values

# Then i padded all data to the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAXLEN = 250

train_padded = pad_sequences(train_sequences, maxlen=MAXLEN)
test_padded = pad_sequences(test_sequences, maxlen=MAXLEN)

# I built the model with 10000 input nodes (of embedding layer) of output dimensions 32, then an LSTM layer to remember context, then finally an output layer
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

# Fitting and training 80 % and testing 20%
history = model.fit(train_padded, train_labels, epochs=10, validation_split=0.2)

model.summary()

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - acc: 0.8598 - loss: 0.3633 - val_acc: 0.9773 - val_loss: 0.1191
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - acc: 0.9770 - loss: 0.0918 - val_acc: 0.9844 - val_loss: 0.0649
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - acc: 0.9847 - loss: 0.0584 - val_acc: 0.9856 - val_loss: 0.0512
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - acc: 0.9893 - loss: 0.0356 - val_acc: 0.9844 - val_loss: 0.0634
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - acc: 0.9929 - loss: 0.0316 - val_acc: 0.9892 - val_loss: 0.0485
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - acc: 0.9958 - loss: 0.0174 - val_acc: 0.9856 - val_loss: 0.0561
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - acc: 0.9979 - loss: 0.0116 - val_acc: 0.9904 - val_loss: 0.0449
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - acc: 0.9974 - loss: 0.0094 - val_acc: 0.9844 - val_loss: 0.0521
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 250, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 656,708 (2.51 MB)

 Trainable params: 328,353 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 328,355 (1.25 MB)

In [8]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
  sequence = tokenizer.texts_to_sequences([pred_text])

  padded = pad_sequences(sequence, maxlen=MAXLEN)
  pred = model.predict(padded)[0][0]  # Extract predicted value from a nested numpy array

  label = 'spam' if pred > 0.5 else 'ham'
  prediction = float(pred), label
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
(0.0001501278456998989, 'ham')


In [9]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
You passed the challenge. Great job!
